In [0]:
import pyspark.sql.functions as F

catalogo = "medalhao"
silver_db_name = "silver"
gold_db_name = "gold"

#  Área de Logística (Vendas por Localidade)

1.1 Criação da tabela gold.ft_vendas_consumidor_local

In [0]:
df_pedido_total = spark.table(f"{catalogo}.{silver_db_name}.ft_pedido_total")
df_consumidores = spark.table(f"{catalogo}.{silver_db_name}.ft_consumidores")


In [0]:
display(df_pedido_total.limit(10))
df_pedido_total.printSchema()
display(df_consumidores.limit(10))
df_consumidores.printSchema()

In [0]:
df_pedido_total_sel = df_pedido_total.select(
    "id_pedido",
    "id_consumidor",
    F.col("valor_total_pago_brl").alias("valor_total_pedido_brl"),
    "data_pedido"
)

df_consumidores_sel = df_consumidores.select(
    "id_consumidor",
    "cidade",
    "estado"
)

In [0]:
df_vendas_consumidor_local = (
    df_pedido_total_sel
        .join(df_consumidores_sel, "id_consumidor", "left")
        .withColumn("data_ingestao", F.current_timestamp()) 
)


display(df_vendas_consumidor_local.limit(10))

In [0]:
(
    df_vendas_consumidor_local
        .select(
            "id_pedido",
            "id_consumidor",
            "valor_total_pedido_brl",   
            "cidade",
            "estado",
            "data_pedido",
            "data_ingestao"
        )
        .write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")  
        .saveAsTable(f"{catalogo}.{gold_db_name}.ft_vendas_consumidor_local")
)

In [0]:
display(spark.table(f"{catalogo}.{gold_db_name}.ft_vendas_consumidor_local").limit(10))


1.2 Criação da view gold.view_total_compras_por_consumidor

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_total_compras_por_consumidor AS
SELECT
    cidade,
    estado,
    COUNT(DISTINCT id_pedido) AS quantidade_vendas,
    SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM medalhao.gold.ft_vendas_consumidor_local
GROUP BY cidade, estado;

SELECT *
FROM medalhao.gold.view_total_compras_por_consumidor
ORDER BY valor_total_localidade DESC 
LIMIT 10;

# 2º Projeto — Área de Logística (Análise de Atrasos de Entregas)


2.1 Criação da tabela gold.ft_atrasos_pedidos_local_vendedor

In [0]:
df_pedidos = spark.table(f"{catalogo}.{silver_db_name}.ft_pedidos")
df_consumidores = spark.table(f"{catalogo}.{silver_db_name}.ft_consumidores")
df_itens = spark.table(f"{catalogo}.{silver_db_name}.ft_itens_pedidos")

display(df_pedidos.limit(10))
df_pedidos.printSchema()

display(df_consumidores.limit(10))
df_consumidores.printSchema()

display(df_itens.limit(10))
df_itens.printSchema()

In [0]:
df_pedidos_sel = df_pedidos.select(
    "id_pedido",
    "id_consumidor",
    "entrega_no_prazo",
    "tempo_entrega_dias",
    "tempo_entrega_estimado_dias"
)

df_consumidores_sel = df_consumidores.select(
    "id_consumidor",
    "cidade",
    "estado"
)

df_itens_sel = df_itens.select(
    "id_pedido",
    "id_vendedor"
)

df_atrasos = (
    df_pedidos_sel
        .join(df_itens_sel, "id_pedido", "left")
        .join(df_consumidores_sel, "id_consumidor", "left")
        .withColumn("data_ingestao", F.current_timestamp()) 
)


In [0]:
(
    df_atrasos
        .select(
            "id_pedido",
            "id_vendedor",
            "id_consumidor",
            "entrega_no_prazo",
            "tempo_entrega_dias",
            "tempo_entrega_estimado_dias",
            "cidade",
            "estado",
            "data_ingestao"
        )
        .write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(f"{catalogo}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor")
)

display(spark.table(f"{catalogo}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor").limit(10))




2.2 Criação das Views Analíticas

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_tempo_medio_entrega_localidade AS
SELECT
    cidade,
    estado,
    AVG(tempo_entrega_dias) AS tempo_medio_entrega,
    AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado,
    CASE 
        WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) 
            THEN 'SIM'
        ELSE 'NÃO'
    END AS entrega_maior_que_estimado
FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY cidade, estado;

SELECT *
FROM medalhao.gold.view_tempo_medio_entrega_localidade
ORDER BY tempo_medio_entrega DESC
LIMIT 50;


In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendedor_pontualidade AS
SELECT
    id_vendedor,
    COUNT(*) AS total_pedidos,
    SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) AS total_atrasados,
    ROUND(
        SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) 
        / COUNT(*) * 100,
        2
    ) AS percentual_atraso
FROM medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY id_vendedor;

SELECT *
FROM medalhao.gold.view_vendedor_pontualidade
LIMIT 50;

# 3º Projeto — Área Comercial (Análises de Vendas por Período)

3.1 Criação da Dimensão de Tempo — gold.dm_tempo

In [0]:
%sql
SELECT 
    MIN(data_pedido) AS data_min,
    MAX(data_pedido) AS data_max
FROM medalhao.silver.ft_pedido_total;

In [0]:
%sql
CREATE OR REPLACE TABLE medalhao.gold.dm_tempo AS
WITH calendario AS (
    SELECT explode(
        sequence(
            to_date('2016-09-04'),   
            to_date('2018-10-17'), 
            interval 1 day
        )
    ) AS sk_tempo
)

SELECT
    sk_tempo,
    year(sk_tempo) AS ano,
    quarter(sk_tempo) AS trimestre,
    month(sk_tempo) AS mes,
    weekofyear(sk_tempo) AS semana_do_ano,
    day(sk_tempo) AS dia,
    dayofweek(sk_tempo) AS dia_da_semana_num,
    date_format(sk_tempo, 'EEEE') AS dia_da_semana_nome,
    date_format(sk_tempo, 'MMMM') AS mes_nome,
    CASE 
        WHEN dayofweek(sk_tempo) IN (1,7) THEN 'Sim'
        ELSE 'Não'
    END AS eh_fim_de_semana
FROM calendario;



In [0]:
%sql
SELECT * FROM medalhao.gold.dm_tempo LIMIT 80;


3.2 Criação da Fato gold.ft_vendas_geral

In [0]:
%sql
CREATE OR REPLACE TABLE medalhao.gold.ft_vendas_geral AS
SELECT
    --IDs
    itens.id_pedido,
    itens.id_item,

    --FKs
    pedidos.id_consumidor AS fk_cliente,
    itens.id_produto AS fk_produto,
    itens.id_vendedor AS fk_vendedor,
    DATE(pedidos.pedido_compra_timestamp) AS fk_tempo,

    --staus/logistica
    pedidos.status_original AS status_pedido,
    pedidos.tempo_entrega_dias,
    pedidos.entrega_no_prazo,

    --itens valores
    itens.preco_brl AS valor_produto_brl,
    itens.preco_frete AS valor_frete_brl,
    (itens.preco_brl + itens.preco_frete) AS valor_total_item_brl,
    ROUND(itens.preco_brl / cot.cotacao_dolar, 2) AS valor_produto_usd,
    ROUND(itens.preco_frete / cot.cotacao_dolar, 2) AS valor_frete_usd,
    ROUND((itens.preco_brl + itens.preco_frete) / cot.cotacao_dolar, 2) AS valor_total_item_usd,
    cot.cotacao_dolar,

    aval.avaliacao AS avaliacao_pedido

FROM medalhao.silver.ft_itens_pedidos AS itens

LEFT JOIN medalhao.silver.ft_pedidos AS pedidos
    ON itens.id_pedido = pedidos.id_pedido

LEFT JOIN medalhao.silver.dm_cotacao_dolar AS cot
    ON DATE(pedidos.pedido_compra_timestamp) = cot.data

LEFT JOIN medalhao.silver.ft_avaliacoes_pedidos AS aval
    ON itens.id_pedido = aval.id_pedido;



In [0]:
%sql
SELECT *
FROM medalhao.gold.ft_vendas_geral
LIMIT 20;

3.3 Criação da view gold.view_vendas_por_periodo

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_por_periodo AS
SELECT
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num,

    COUNT(DISTINCT f.id_pedido) AS total_pedidos,
    COUNT(*) AS total_itens,
    SUM(f.valor_total_item_brl) AS receita_total_brl,
    SUM(f.valor_total_item_usd) AS receita_total_usd,
    ROUND(AVG(f.valor_total_item_brl), 2) AS ticket_medio_brl,
    ROUND(AVG(f.avaliacao_pedido), 2) AS avaliacao_media

FROM medalhao.gold.ft_vendas_geral AS f

INNER JOIN medalhao.gold.dm_tempo AS t
    ON f.fk_tempo = t.sk_tempo

GROUP BY
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num;


In [0]:
%sql
SELECT *
FROM medalhao.gold.view_vendas_por_periodo
ORDER BY ano, mes, dia
LIMIT 20;


Qual é o dia da semana com maior receita total em reais?

In [0]:
%sql
SELECT 
    dia_da_semana_num,
    SUM(receita_total_brl) AS receita_total
FROM medalhao.gold.view_vendas_por_periodo
GROUP BY dia_da_semana_num
ORDER BY receita_total DESC
LIMIT 1;


No último ano disponível, qual mês teve o maior ticket médio?

In [0]:
%sql
WITH ultimo_ano AS (
    SELECT MAX(ano) AS ano
    FROM medalhao.gold.view_vendas_por_periodo
)

SELECT
    v.mes,
    v.mes_nome,
    v.ticket_medio_brl
FROM medalhao.gold.view_vendas_por_periodo v
JOIN ultimo_ano u
    ON v.ano = u.ano
GROUP BY v.mes, v.mes_nome, v.ticket_medio_brl
ORDER BY v.ticket_medio_brl DESC
LIMIT 1;


3.4. Criação da gold.view_top_produto


In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_top_produto AS
SELECT
    f.fk_produto AS id_produto,
    p.categoria_produto AS categoria_produto,

    -- MÉTRICAS DE VENDAS
    COUNT(*) AS quantidade_vendida,
    COUNT(DISTINCT f.id_pedido) AS total_pedidos,

    SUM(f.valor_total_item_brl) AS receita_brl,
    SUM(f.valor_total_item_usd) AS receita_usd,

    ROUND(AVG(f.valor_produto_brl), 2) AS preco_medio_brl,
    ROUND(AVG(f.avaliacao_pedido), 2) AS avaliacao_media,

    -- PESO 
    ROUND(AVG(p.peso_produto_gramas), 2) AS peso_medio_gramas

FROM medalhao.gold.ft_vendas_geral f

LEFT JOIN medalhao.silver.ft_produtos p
    ON f.fk_produto = p.id_produto

GROUP BY
    f.fk_produto,
    p.categoria_produto;


In [0]:
%sql
SELECT *
FROM medalhao.gold.view_top_produto
ORDER BY receita_brl DESC
LIMIT 20;


3.5 Criação da view_vendas_produtos_esteticos

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_produtos_esteticos AS

WITH produtos_fashion AS (
    SELECT
        p.id_produto,
        p.categoria_produto
    FROM medalhao.silver.ft_produtos p
    WHERE p.categoria_produto LIKE 'fashion%'    
)

SELECT
    t.ano,
    t.mes,
    pf.categoria_produto,

    COUNT(DISTINCT f.id_pedido) AS total_pedidos,
    COUNT(*) AS total_itens_vendidos,

    SUM(f.valor_total_item_brl) AS receita_total_brl,
    SUM(f.valor_total_item_usd) AS receita_total_usd,

    ROUND(AVG(f.valor_total_item_brl), 2) AS ticket_medio_brl,
    ROUND(AVG(f.valor_total_item_usd), 2) AS ticket_medio_usd,

    ROUND(AVG(f.avaliacao_pedido), 2) AS avaliacao_media

FROM medalhao.gold.ft_vendas_geral f

INNER JOIN produtos_fashion pf
    ON f.fk_produto = pf.id_produto

INNER JOIN medalhao.gold.dm_tempo t
    ON f.fk_tempo = t.sk_tempo

GROUP BY
    t.ano,
    t.mes,
    pf.categoria_produto;


In [0]:
%sql
SELECT *
FROM medalhao.gold.view_vendas_produtos_esteticos
ORDER BY ano, mes, categoria_produto
LIMIT 30;
